Nonlinear Correlation Information Entropy (QNCIE) :  Q < 1 ===>  POOR Image fusion quality

In [2]:
import cv2
import numpy as np
from skimage.measure import shannon_entropy

img1_path = "/Users/chintubharath/Documents/sdp/Imfusion-main/demo/medical1.png"
img2_path = "/Users/chintubharath/Documents/sdp/Imfusion-main/demo/medical2.png"


def information_entropy(image):
    return shannon_entropy(image)

def joint_information_entropy(image1, image2):
    joint_image = np.dstack((image1, image2))
    return shannon_entropy(joint_image.reshape(-1, joint_image.shape[-1]))

def QNCIE(imageA, imageB, imageF):
    # Resize images to have the same dimensions
    imageA = cv2.resize(imageA, (imageF.shape[1], imageF.shape[0]))
    imageB = cv2.resize(imageB, (imageF.shape[1], imageF.shape[0]))
    
    # Calculate information entropy of each image
    entropy_A = information_entropy(imageA)
    entropy_B = information_entropy(imageB)
    entropy_F = information_entropy(imageF)
    
    # Calculate joint information entropy
    joint_AB = joint_information_entropy(imageA, imageB)
    joint_AF = joint_information_entropy(imageA, imageF)
    joint_BF = joint_information_entropy(imageB, imageF)
    
    # Calculate QNCIE
    QNCIE_AB = entropy_A + entropy_B - joint_AB
    QNCIE_AF = entropy_A + entropy_F - joint_AF
    QNCIE_BF = entropy_B + entropy_F - joint_BF
    
    return QNCIE_AB, QNCIE_AF, QNCIE_BF

# Load the images
imageA = cv2.imread(img1_path, cv2.IMREAD_GRAYSCALE)
imageB = cv2.imread(img2_path, cv2.IMREAD_GRAYSCALE)
imageF = cv2.imread('/Users/chintubharath/Documents/sdp/outputs/fusion-1.png', cv2.IMREAD_GRAYSCALE)

# Calculate QNCIE
QNCIE_AB, QNCIE_AF, QNCIE_BF = QNCIE(imageA, imageB, imageF)

print("QNCIE between image A and image B:", QNCIE_AB)
print("QNCIE between image A and fusion image F:", QNCIE_AF)
print("QNCIE between image B and fusion image F:", QNCIE_BF)


QNCIE between image A and image B: 4.638776449327568
QNCIE between image A and fusion image F: 4.712412866494064
QNCIE between image B and fusion image F: 5.696567297462054


Mutual Information (MI)

In [5]:
epsilon = 1e-10  # Small constant to avoid division by zero

def mutual_information(imageA, imageB, imageF):
    # Compute edge histograms
    edge_hist_A = edge_histogram(imageA)
    edge_hist_B = edge_histogram(imageB)
    edge_hist_F = edge_histogram(imageF)
    
    # Resize images to have the same dimensions
    imageA = cv2.resize(imageA, (imageF.shape[1], imageF.shape[0]))
    imageB = cv2.resize(imageB, (imageF.shape[1], imageF.shape[0]))
    
    # Compute joint histograms
    joint_hist_AF = joint_histogram(imageA, imageF)
    joint_hist_BF = joint_histogram(imageB, imageF)
    
    # Compute mutual information terms
    term_A_B = np.sum(joint_histogram(imageA, imageB) * np.log2(np.divide(joint_histogram(imageA, imageB) + epsilon, np.outer(edge_histogram(imageA) + epsilon, edge_histogram(imageB) + epsilon))))
    term_A_F = np.sum(joint_hist_AF * np.log2(np.divide(joint_hist_AF + epsilon, np.outer(edge_hist_A + epsilon, edge_hist_F + epsilon))))
    term_B_F = np.sum(joint_hist_BF * np.log2(np.divide(joint_hist_BF + epsilon, np.outer(edge_hist_B + epsilon, edge_hist_F + epsilon))))
    
    # Compute mutual information
    MI_AB = term_A_B
    MI_AF = term_A_F
    MI_BF = term_B_F
    
    return MI_AB, MI_AF, MI_BF

# Calculate mutual information
MI_AB, MI_AF, MI_BF = mutual_information(imageA, imageB, imageF)
print("Mutual Information (MI) between image A and image B:", MI_AB)
print("Mutual Information (MI) between image A and fused image F:", MI_AF)
print("Mutual Information (MI) between image B and fused image F:", MI_BF)


Mutual Information (MI) between image A and image B: 35.10670639065237
Mutual Information (MI) between image A and fused image F: 34.58568445508886
Mutual Information (MI) between image B and fused image F: 48.59555380585328


/var/folders/1c/wgg3jchj1vnb_c49fv64srjm0000gn/T/ipykernel_17317/554599845.py:11: RuntimeWarning: Converting input from bool to <class 'numpy.uint8'> for compatibility.
  hist, _ = np.histogram(edges, bins=256, range=(0, 1))


Feature Mutual Information (FMI) 

In [6]:
def feature_map(image):
    # You can use any feature extraction method here
    # For example, you can compute the histogram of oriented gradients (HOG)
    # Here, let's use edge histogram as a simple feature map
    return edge_histogram(image)

def feature_mutual_information(imageA, imageB, imageF):
    # Compute feature maps
    feature_map_A = feature_map(imageA)
    feature_map_B = feature_map(imageB)
    feature_map_F = feature_map(imageF)
    
    # Compute Mutual Information (MI)
    MI_AB, MI_AF, MI_BF = mutual_information(imageA, imageB, imageF)
    
    # Compute Feature Mutual Information (FMI)
    FMI = MI_AB + MI_AF + MI_BF
    
    return FMI

# Calculate Feature Mutual Information (FMI)
FMI = feature_mutual_information(imageA, imageB, imageF)
print("Feature Mutual Information (FMI):", FMI)


Feature Mutual Information (FMI): 118.2879446515945


/var/folders/1c/wgg3jchj1vnb_c49fv64srjm0000gn/T/ipykernel_17317/554599845.py:11: RuntimeWarning: Converting input from bool to <class 'numpy.uint8'> for compatibility.
  hist, _ = np.histogram(edges, bins=256, range=(0, 1))


Normalized Mutual Information (NMI)

In [7]:
def entropy(image):
    # Compute entropy of the image
    hist, _ = np.histogram(image.ravel(), bins=256, range=(0, 255))
    hist = hist / np.sum(hist)  # Normalize histogram
    entropy = -np.sum(hist * np.log2(hist + 1e-10))  # Add epsilon to avoid log(0)
    return entropy

def normalized_mutual_information(imageA, imageB, imageF):
    # Compute entropies
    entropy_A = entropy(imageA)
    entropy_B = entropy(imageB)
    entropy_F = entropy(imageF)
    
    # Compute Mutual Information (MI)
    MI_AB, MI_AF, MI_BF = mutual_information(imageA, imageB, imageF)
    
    # Compute Normalized Mutual Information (NMI)
    NMI = 2 * ((MI_AB / (entropy_A + entropy_B)) + (MI_AF / (entropy_A + entropy_F)) + (MI_BF / (entropy_B + entropy_F)))
    
    return NMI

# Calculate Normalized Mutual Information (NMI)
NMI = normalized_mutual_information(imageA, imageB, imageF)
print("Normalized Mutual Information (NMI):", NMI)


Normalized Mutual Information (NMI): 22.33800349568718


/var/folders/1c/wgg3jchj1vnb_c49fv64srjm0000gn/T/ipykernel_17317/554599845.py:11: RuntimeWarning: Converting input from bool to <class 'numpy.uint8'> for compatibility.
  hist, _ = np.histogram(edges, bins=256, range=(0, 1))


In [9]:
def mean_square_error(imageA, imageB, imageF):
    # Resize images to have the same dimensions
    imageA = cv2.resize(imageA, (imageF.shape[1], imageF.shape[0]))
    imageB = cv2.resize(imageB, (imageF.shape[1], imageF.shape[0]))
    
    # Compute Mean Square Error (MSE) between image A and fused image F
    mse_AF = np.mean((imageA - imageF) ** 2)
    
    # Compute Mean Square Error (MSE) between image B and fused image F
    mse_BF = np.mean((imageB - imageF) ** 2)
    
    # Compute the average of MSE values
    mse = (mse_AF + mse_BF) / 2
    
    return mse

# Calculate Mean Square Error (MSE)
MSE = mean_square_error(imageA, imageB, imageF)
print("Mean Square Error (MSE):", MSE)


Mean Square Error (MSE): 90.11621966794381


Root Mean Square Error (RMSE)

The Root Mean Square Error (RMSE) between the reference images and the fused image is approximately 9.49.

This value represents the square root of the average squared difference between corresponding pixel values in the reference images (A and B) and the fused image (F). A lower RMSE value indicates a closer resemblance between the fused image and the source images, suggesting a better fusion effect.

Just like Mean Square Error (MSE), the interpretation of RMSE values depends on the scale and characteristics of the images. Always consider other evaluation metrics alongside RMSE for a comprehensive assessment of fusion quality.

In [12]:
def root_mean_square_error(imageA, imageB, imageF):
    # Resize images to have the same dimensions
    imageA = cv2.resize(imageA, (imageF.shape[1], imageF.shape[0]))
    imageB = cv2.resize(imageB, (imageF.shape[1], imageF.shape[0]))
    
    # Compute Mean Square Error (MSE) between image A and fused image F
    mse_AF = np.mean((imageA - imageF) ** 2)
    
    # Compute Mean Square Error (MSE) between image B and fused image F
    mse_BF = np.mean((imageB - imageF) ** 2)
    
    # Compute the average of MSE values
    mse = (mse_AF + mse_BF) / 2
    
    # Compute the Root Mean Square Error (RMSE)
    rmse = np.sqrt(mse)
    
    return rmse

# Calculate Root Mean Square Error (RMSE)
RMSE = root_mean_square_error(imageA, imageB, imageF)
print("Root Mean Square Error (RMSE):", RMSE)


Root Mean Square Error (RMSE): 9.492956318657734
